In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages as pdf
import numpy as np

import sys
import requests 
import urllib.request
import json
import importlib
import os
import datetime as dt

base_path = os.getcwd()
sys.path.append(base_path + '//..')

import Cumminsaceite as cu
#connection=cu.Connection('camilo.rodriguez.lazo@cummins.cl','camilo2021')
connection=cu.Connection("mauricio.vera@cummins.cl","ctJy.S8Juq8F")##Agregar usuario y clave
# connection=cu.Connection('wilson.soriano@cummins.cl','N<46MfH~-MrF')
connection.getToken()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Se guardara en la carpeta '02_Archivos' 
carpeta_guardado = 'DatosLimites'
save_path = os.path.abspath(base_path + '/../../02_Archivos/' + carpeta_guardado)

if carpeta_guardado:
        # carpeta_guardado = carpeta_guardado + '\\'
        # directory = save_path + '\\' + carpeta_guardado 
        if not os.path.exists(save_path):
            os.makedirs(save_path)

In [3]:
datos_limites = connection.getScaaLimites()

In [4]:
datos_limites.faena.unique()

array(['Quebrada Blanca', 'Andina', 'Los Pelambres', 'Antucoya',
       'Caserones', 'Centinela', 'Cerro Negro Norte', 'Collahuasi',
       'Gaby', 'Ministro Hales', 'Radomiro Tomic', 'Sierra Gorda',
       'Chuquicamata', 'Spence', 'Escondida', 'Lomas Bayas', 'Pto.Infl.',
       'Chuquicamanta'], dtype=object)

In [5]:
limites = datos_limites.copy()

# Filtro
limites.drop(columns=['estado'], inplace=True)




# Filtrar limites por los elementos de un analisis de aceite
elementos_aceite = ['Visc.40','Visc 100','Diluc.','Agua','TBN','Hollin','Oxid.','Nitrac.','Pto.Infl.',
                    'Sulf.','Ind.PQ.','AL','B','CA','CR','CU','FE','K','MG','MO','NA','NI',
                    'P','PB','SI','SN','ZN']

limites = limites[limites['elemento'].isin(elementos_aceite)]

# Orden
limites.sort_values(by=['faena','flota','elemento','desde'], inplace = True) 

limites.reset_index(drop=True, inplace=True)

unidad_elementos = limites.groupby('elemento')['unidadmedida'].agg(pd.Series.mode)

limites.drop(columns=['unidadmedida'], inplace=True)

In [6]:
unidad_elementos.to_json(save_path + '\\unidades.json')

In [7]:
limites.to_csv(save_path + '\\limites.csv', index=False)

In [8]:
temp = limites[limites.faena == 'Andina']
temp = temp[temp.flota == '930 E']
temp[temp.elemento == 'AL']

,faena,flota,elemento,limiteprecaucion,limitecritico,desde,hasta
0,Andina,930 E,AL,10.0,15.0,1980-01-01,2009-07-05
1,Andina,930 E,AL,10.0,15.0,2009-07-05,2014-05-29
2,Andina,930 E,AL,15.0,30.0,2009-07-05,2014-05-29
3,Andina,930 E,AL,15.0,30.0,2014-05-29,2017-05-16
4,Andina,930 E,AL,5.0,10.0,2014-05-29,2017-05-16
5,Andina,930 E,AL,5.0,10.0,2017-05-16,2018-06-14
6,Andina,930 E,AL,4.0,5.0,2017-05-16,2018-06-14
7,Andina,930 E,AL,4.0,5.0,2018-06-14,2018-11-20
8,Andina,930 E,AL,4.3,5.2,2018-06-14,2018-11-20
9,Andina,930 E,AL,4.3,5.2,2018-11-20,2020-01-01


In [9]:
limites[limites.faena=='Chuquicamata']

,faena,flota,elemento,limiteprecaucion,limitecritico,desde,hasta
1950,Chuquicamata,PC 5500,AL,5.0,7.0,1980-01-01,2013-07-01
1951,Chuquicamata,PC 5500,AL,5.0,7.0,2013-07-01,2019-01-02
1952,Chuquicamata,PC 5500,AL,5.0,10.0,2013-07-01,2019-01-02
1953,Chuquicamata,PC 5500,AL,5.0,10.0,2019-01-02,2020-01-01
1954,Chuquicamata,PC 5500,AL,5.0,10.0,2019-01-02,2020-01-01
...,...,...,...,...,...,...,...
2069,Chuquicamata,PC 5500,Visc.40,89.0,115.0,2013-07-01,2019-01-02
2070,Chuquicamata,PC 5500,Visc.40,87.0,130.0,2013-07-01,2019-01-02
2071,Chuquicamata,PC 5500,Visc.40,87.0,130.0,2019-01-02,2020-01-01
2072,Chuquicamata,PC 5500,Visc.40,87.0,130.0,2019-01-02,2020-01-01
